# 加入出发地当天天气信息

In [1]:
import pandas as pd

In [2]:
# 创建出发地天气id字典
airport = pd.read_csv('./dataset/temp_airport.csv', encoding= 'gbk')
# 读取airport.csv文件为字典
weather_dict = {}
for i in range(len(airport)):
    weather_dict[airport['机场编码'][i]] = airport['网站ID'][i]
print(weather_dict)

{'JDZ': 58527, 'PVG': 58362, 'TGO': 54135, 'XMN': 59134, 'URC': 51463, 'TYN': 53772, 'SZX': 59493, 'SYX': 59948, 'TAO': 54857, 'TNA': 54823, 'SJW': 53698, 'SHE': 54342, 'SHA': 58362, 'PEK': 54511, 'NKG': 58238, 'KWL': 57957, 'KMG': 56778, 'HRB': 50953, 'HET': 53463, 'HGH': 58457, 'FOC': 58847, 'CTU': 57516, 'CKG': 57516, 'HFE': 58321, 'CGO': 57083, 'TLQ': 51573, 'WUH': 57494}


# 进入网站爬取数据


In [3]:
#将id变为str
from Write import write
for city in weather_dict:
    id = str(weather_dict[city])
    write(city, id)
    print(city, '(', id, ')已写入csv文件')

http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=01&y1=2015&d2=28&m2=01&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=02&y1=2015&d2=28&m2=02&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=03&y1=2015&d2=28&m2=03&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=04&y1=2015&d2=28&m2=04&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=05&y1=2015&d2=28&m2=05&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=06&y1=2015&d2=28&m2=06&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=07&y1=2015&d2=28&m2=07&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=08&y1=2015&d2=28&m2=08&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=09&y1=2015&d2=28&m2=09&y2=2015
写入完成
http://www.meteomanz.com/sy2?l=1&cou=2250&ind=58527&d1=01&m1=10&y1=2015&d2=28&m2=10&y2=2015
写入完成
http://www.meteomanz.com/sy2?l